## Prediction of Medical Provider Fraud

# Problem Statement :

 - ### Predict the probability that a provider is potential fraud  and flag them accordingly.

 - ### Detect Bias in Pre Train & Post Train Dataset.

 - ### Detect the important predictors which are impacting the final prediction 
 
 - ###  Explain the final prediction using Exlainable AI modules - SHAPley, LIME, What-If 



### Mounting the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

### Importing the libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import warnings     # for supressing a warning when importing large files
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,roc_auc_score,classification_report
import pickle
from scipy import stats
import time
from sklearn.model_selection import GridSearchCV,KFold

from pylab import rcParams

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42

import os


import matplotlib.pyplot as plt
import seaborn as sns


# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

### Load the training data

In [ ]:
cms_prov_drug = pd.read_csv("/content/drive/MyDrive/CMS/Medicare_Part_D_Prescribers_by_Provider_and_Drug_2019_12_31.csv")

cms_train154 = pd.read_csv("/content/drive/MyDrive/CMS/Train-1542865627584.csv")

cms_train_bene = pd.read_csv("/content/drive/MyDrive/CMS/Train_Beneficiarydata-1542865627584.csv")

cms_train_inpatient = pd.read_csv("/content/drive/MyDrive/CMS/Train_Inpatientdata-1542865627584.csv")
cms_train_outpatient = pd.read_csv("/content/drive/MyDrive/CMS/Train_Outpatientdata-1542865627584.csv")

#cms_train154 = pd.read_csv("Train-1542865627584.csv")

#cms_train_bene = pd.read_csv("Train_Beneficiarydata-1542865627584.csv")

#cms_train_inpatient = pd.read_csv("Train_Inpatientdata-1542865627584.csv")
#cms_train_outpatient = pd.read_csv("Train_Outpatientdata-1542865627584.csv")


In [ ]:
print("cms_train154.shape: ",cms_train154.shape  )
print("cms_train_bene.shape :", cms_train_bene.shape)
print("cms_train_inpatient.shape: ", cms_train_inpatient.shape)
print("cms_train_outpatient.shape: ", cms_train_outpatient.shape)

## LOADING TEST DATA

In [ ]:
cms_test154 = pd.read_csv("/content/drive/MyDrive/CMS/Test-1542969243754.csv")
cms_test_bene = pd.read_csv("/content/drive/MyDrive/CMS/Test_Beneficiarydata-1542969243754.csv")
cms_test_inpatient = pd.read_csv("/content/drive/MyDrive/CMS/Test_Inpatientdata-1542969243754.csv")
cms_test_outpatient = pd.read_csv("/content/drive/MyDrive/CMS/Test_Outpatientdata-1542969243754.csv")

#cms_test154 = pd.read_csv("Test-1542969243754.csv")
#cms_test_bene = pd.read_csv("Test_Beneficiarydata-1542969243754.csv")
#cms_test_inpatient = pd.read_csv("Test_Inpatientdata-1542969243754.csv")
#cms_test_outpatient = pd.read_csv("Test_Outpatientdata-1542969243754.csv")

In [ ]:

print("cms_test154.shape: ",cms_test154.shape  )
print("cms_test_bene.shape :", cms_test_bene.shape)
print("cms_test_inpatient.shape: ", cms_test_inpatient.shape)
print("cms_test_outpatient.shape: ", cms_test_outpatient.shape)

### Merge Train & Test data

In [ ]:
# Merge Inpatient, Outpatient and beneficiary dataframe into a single patient dataset
cms_provider = pd.merge(cms_train154, cms_test154,how = 'outer', on = 'Provider')
cms_provider = pd.concat([cms_train154, cms_test154], axis=0)
print(cms_provider.shape)


                     

In [ ]:
cms_bene = pd.merge(cms_train_bene, cms_test_bene,how = 'outer', on = 'BeneID')

cms_bene = pd.concat([cms_train_bene, cms_test_bene], axis=0)
print(cms_bene.shape)

In [ ]:
#cms_inpatient = pd.merge(cms_train_inpatient, cms_test_inpatient,how = 'outer', on = ['BeneID', 'ClaimID'])
cms_inpatient = pd.concat([cms_train_inpatient, cms_test_inpatient], axis=0)
print(cms_inpatient.shape)

In [ ]:
#cms_outpatient = pd.merge(cms_train_outpatient, cms_test_outpatient,how = 'outer', on = ['BeneID', 'ClaimID'])
cms_outpatient = pd.concat([cms_train_outpatient, cms_test_outpatient], axis=0)
print(cms_outpatient.shape)


#### Missing value in cms_provider is due to Test data where Label data "PotentialFraud" is not present

In [ ]:
cms_provider.info()

In [ ]:
cms_bene.info()

#### Benficiary dataset -> Only Missing value is DOD column

### Derive Inpatient & OutPatient Indicator column which will be used in marged data to detect patient type

In [ ]:
#Function to calculate the unique count of columns for rach row
def N_unique_values(df):
    return np.array([len(set([i for i in x[~pd.isnull(x)]])) for x in df.values])

#### Inpatient & Putpatient dataset have multiple missing values.

In [ ]:
cms_inpatient["IsInpatient"] = 1
cms_outpatient["IsInpatient"] = 0

### Analyze the categorical values in beneficiary dataset

In [ ]:
# subset all categorical variables
cms_bene_categorical = cms_bene.select_dtypes(include=['object'])
cms_bene_categorical.columns

In [ ]:
colist= [ 'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke']
for i in colist :
  print(cms_bene[i].value_counts())

### Categorical variables in cms_train_bene file are having values 1 & 2.
2 means Chronic disease = "No" and 1 means "Yes".

### Let's replace the 2 with Zero. Hence after the mapping ->

### chronic condition :  "NO" is 0 and "YES" is 1

#### No dummy is required as they have only binary categorization


In [ ]:
cms_bene = cms_train_bene.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                           'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2, 
                           'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2, 
                           'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2 }, 0)

cms_bene = cms_bene.replace({'RenalDiseaseIndicator': 'Y'}, 1)

### Derive age from DOB & DOD in cms_train_bene file 
 
#### Treating the Missing DOD : Missing DOD means the person is alive and fill the missing values with the year when this data was extracted.

#### The last DOD value is 2009-12-01 ,which means Beneficiary Details data was extracted on 2009.
#### Hence fill the mising DOD with 2009





In [ ]:
#Creating a new variable 'IsDead' with flag 1 means Dead and 0 means not Dead

cms_bene.loc[cms_bene.DOD.isna(),'IsDead']=0
cms_bene.loc[cms_bene.DOD.notna(),'IsDead']=1
cms_bene.loc[:,'IsDead'].head(7)




### Derive new columns for in Patient data

#### Deriving days of admission in hospital

In [ ]:
## As patient can be admitted for only for 1 day,lets add 1 to the difference of Discharge Date and Admission Date 

cms_inpatient['AdmissionDt'] = pd.to_datetime(cms_inpatient['AdmissionDt'] , format = '%Y-%m-%d')
cms_inpatient['DischargeDt'] = pd.to_datetime(cms_inpatient['DischargeDt'],format = '%Y-%m-%d')
cms_inpatient['AdmissionDuration'] = ((cms_inpatient['DischargeDt'] - cms_inpatient['AdmissionDt']).dt.days)+1


In [ ]:
cms_inpatient['MismatchedAdmitCode'] = np.where(cms_inpatient['ClmAdmitDiagnosisCode'] == cms_inpatient['ClmDiagnosisCode_1'], 0, 1)

###  Merge the Benefitiary data with in patient & out patient data

In [ ]:
cms_patient_mergeid = [i for i in cms_outpatient.columns if i in cms_inpatient.columns]

# Merge Inpatient, Outpatient and beneficiary dataframe into a single patient dataset
cms_patient_data = pd.merge(cms_inpatient, cms_outpatient,
                   left_on = cms_patient_mergeid,
                   right_on = cms_patient_mergeid,
                   how = 'outer').\
          merge(cms_bene,left_on='BeneID',right_on='BeneID',how='inner')

In [ ]:
cms_patient_data.columns

### Missing value treatment on merged beneficiary & in/out patient data

In [ ]:

missing_columns = cms_patient_data.columns[cms_patient_data.isnull().sum() > 1]
missing_columns

In [ ]:
cms_bene['DOB'] = pd.to_datetime(cms_bene['DOB'] , format = '%Y-%m-%d')
cms_bene['DOD'] = pd.to_datetime(cms_bene['DOD'],format = '%Y-%m-%d',errors='ignore')
#cms_bene['age'] = round(((cms_bene['DOD'] - cms_bene['DOB']).dt.days)/365)


cms_patient_data['age'] = pd.to_datetime(cms_patient_data['ClaimStartDt']) - pd.to_datetime(cms_patient_data['DOB'])
cms_patient_data['age'] = np.floor(cms_patient_data['age'].astype(str).str.extract('(\d+)').astype(int) / 365.24)



cms_patient_data['Age_Under_65'] = np.where(cms_patient_data.age < 65, 1, 0)

cms_patient_data['Age_65_upto_75'] = np.where((cms_patient_data.age >= 65) & (cms_patient_data.age < 75), 1, 0)
cms_patient_data['Age_75_upto_85'] = np.where((cms_patient_data.age >= 75) & (cms_patient_data.age < 85), 1, 0)

cms_patient_data['Age_85_or_Over'] = np.where(cms_patient_data.age >= 85, 1, 0)

cms_patient_data.head(5)

In [ ]:
cms_race_dummy = pd.get_dummies(cms_patient_data["Race"], prefix = "Race" )
print(cms_race_dummy.head())

# concat dummy variables with categorical dataset
cms_patient_data = pd.concat([cms_patient_data, cms_race_dummy], axis=1)

In [ ]:
PhysicianList = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']
# Lets's count the number of Physicians for each record and store them in a new variable

cms_patient_data['N_UniquePhysician'] = N_unique_values(cms_patient_data[PhysicianList])


cms_patient_data['N_UniquePhysician'].value_counts()


#### Let's replace AttendingPhysician, OperatingPhysician, OtherPhysician, columns value with 0 and 1 where we have value we are replacing it with 1 and in place of null value we replace it with 0.


In [ ]:


cms_patient_data[['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']] = np.where(cms_patient_data[['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']].isnull(), 0, 1)


### Derive total number of physicians are attached with the claim of the patient

In [ ]:



cms_patient_data['total_physicians'] = cms_patient_data['AttendingPhysician'] +  cms_patient_data['OperatingPhysician'] + cms_patient_data['OtherPhysician']

cms_patient_data['total_physicians']

In [ ]:
#cms_patient_data[cms_patient_data['BeneID'] == 'BENE100002']



# Treating missing values in DignosisGroupCode Count

In [ ]:

count_DiagnosisGroupCode=cms_patient_data['DiagnosisGroupCode'].value_counts()
count_DiagnosisGroupCode=count_DiagnosisGroupCode[:20] # To show only top 20 codes 
count_DiagnosisGroupCode

#### Since DignosisGroupCode has maximum values as null, so we are s by creating a new column "IsDiagnosisCode" in which value will either "1" or "0" .
#### if groupDiagnosiscode  has null value in a claim then "IsDiagnosisCode" column value is 0 otherwise 1

In [ ]:


cms_patient_data['IsDiagnosisCode'] = np.where(cms_patient_data.DiagnosisGroupCode.notnull(), 1, 0)
#cms_patient_data = cms_patient_data.drop(['DiagnosisGroupCode'], axis = 1) # We are droping the column "DiagnosisGroupCode"

### Handling missing values for "DeductibleamtPaid" column

#### Check number of missing values in this variable

In [ ]:

cms_patient_data['DeductibleAmtPaid'].isnull().sum()  

In [ ]:
## Count Plot of "DeductibleAmtPaid" values

fig=plt.figure(figsize=(10,5))
sns.countplot(cms_patient_data['DeductibleAmtPaid'])

#### maximum values are 0 here

In [ ]:
## Box plot of this "DeductibleAmtPaid", maximum values are 0 that shows here.

fig=plt.figure(figsize=(8,6))
sns.boxplot(cms_patient_data['DeductibleAmtPaid'])
fig.tight_layout()

In [ ]:
## The above analysis tells that we can replace missing values with 0 

cms_patient_data['DeductibleAmtPaid'].fillna(0,inplace=True)

In [ ]:
### creating one new variable "IsDeductibleAmtPaid" which tells is DeductibleAmtPaid Zero or not

cms_patient_data['IsDeductibleAmtPaid']=np.where(cms_patient_data['DeductibleAmtPaid']==0,0,1) 

In [ ]:
fig=plt.figure(figsize=(8,6))
sns.countplot(cms_patient_data['IsDeductibleAmtPaid'])

print(cms_patient_data['IsDeductibleAmtPaid'].value_counts())

In [ ]:
### Handling missing values for "AdmitForDays" column, Count of missing values in this column

cms_patient_data['AdmissionDuration'].isnull().sum() 

In [ ]:
# Replace all missing value with 0 which indicates patients who was not admitted to hospital

cms_patient_data['AdmissionDuration'].fillna(0,inplace=True)

In [ ]:
cms_patient_data['AdmissionDuration'].isnull().sum() 

In [ ]:
### Count number of records
## From here we get the count of BeneID and ClaimId for each provider

## For Train 
cms_patient_data_cnt = cms_patient_data[['BeneID', 'ClaimID']].groupby(cms_patient_data['Provider']).nunique().reset_index()
cms_patient_data_cnt.rename(columns={'BeneID':'BeneID_count','ClaimID':'ClaimID_count'},inplace=True)

### Handling ClmProcedureCodes variables and missing values


In [ ]:
#Convert ClmProcedure_vars to categorical value
cms_patient_data['ClmProcedureCode_1'] = cms_patient_data['ClmProcedureCode_1'].astype('object')
cms_patient_data['ClmProcedureCode_2'] = cms_patient_data['ClmProcedureCode_2'].astype('object')
cms_patient_data['ClmProcedureCode_3'] = cms_patient_data['ClmProcedureCode_3'].astype('object')
cms_patient_data['ClmProcedureCode_4'] = cms_patient_data['ClmProcedureCode_4'].astype('object')
cms_patient_data['ClmProcedureCode_5'] = cms_patient_data['ClmProcedureCode_5'].astype('object')
cms_patient_data['ClmProcedureCode_6'] = cms_patient_data['ClmProcedureCode_6'].astype('object')

ClmProcedure_vars = ['ClmProcedureCode_{}'.format(x) for x in range(1,7)]
print(ClmProcedure_vars)

In [ ]:
## To Check how many null values are in each Clmprocedurecodes


cms_patient_data[ClmProcedure_vars].isnull().sum()

In [ ]:
cms_patient_data['ClmProcedureCode_1'].value_counts()

### This function helps us find the length of unique values in each row/record

In [ ]:
cms_patient_data[ClmProcedure_vars]


### We count the number of procedureCode for each claim and store these value in a new variable

In [ ]:

cms_patient_data['N_Procedure'] = N_unique_values(cms_patient_data[ClmProcedure_vars])

## So from here we get to know that 534901 claims/records has 0 claim procedure codes, 17820 claims/records has 1 claimprocedurecodes and so on

cms_patient_data['N_Procedure'].value_counts()

### Handling of 'ClmDiagnosisCode'

In [ ]:


# We count the number of claims
ClmDiagnosisCode_vars =['ClmAdmitDiagnosisCode'] + ['ClmDiagnosisCode_{}'.format(x) for x in range(1, 11)]


ClmDiagnosisCode_vars

In [ ]:
# Lets's count the number of CLMDiagnosisCode for each record and store them in a new variable

cms_patient_data['N_UniqueDiagnosis_Claims'] = N_unique_values(cms_patient_data[ClmDiagnosisCode_vars])


cms_patient_data['N_UniqueDiagnosis_Claims'].value_counts()

In [ ]:
cms_patient_data.head(6)

In [ ]:
cms_patient_data["Gender"].value_counts()

In [ ]:
cms_patient_data['Gender']=cms_patient_data['Gender'].replace(2,0)

In [ ]:
cms_patient_data["Gender"].value_counts()

In [ ]:
cms_patient_data.columns

In [ ]:
cms_patient_data["Race"].value_counts()

## Create dummy variables for Race column ->convert into dummies - one hot encoding

In [ ]:
cms_patient_data.columns

In [ ]:
cms_patient_data.shape

In [ ]:
# subset all categorical variables
cms_patient_categorical = cms_patient_data.select_dtypes(include=['object'])
cms_patient_categorical.columns

In [ ]:
## Convert RenalDiseaseIndicator from object to integer

cms_patient_data['RenalDiseaseIndicator']=cms_patient_data.RenalDiseaseIndicator.astype(int) 

fig=plt.figure(figsize=(8,6))
sns.countplot(cms_patient_data['RenalDiseaseIndicator'])
fig.tight_layout()

In [ ]:
cms_provider.columns

In [ ]:
print(cms_patient_data.shape)
print(cms_provider.shape)

In [ ]:
cms_patient_data["Provider"].nunique()

In [ ]:
cms_provider["PotentialFraud"].value_counts()

In [ ]:
cms_test154

In [ ]:
cms_train154["PotentialFraud"].value_counts()

In [ ]:
cms_provider[cms_provider["Provider"] == "PRV57745"]

In [ ]:
cms_provider.info()

In [ ]:
cms_provider = cms_provider.fillna("Test")

In [ ]:
cms_provider["PotentialFraud"].value_counts()

In [ ]:
cms_patient_provider = pd.merge(cms_patient_data, cms_provider, how = "left", on = "Provider")

In [ ]:
print(cms_patient_provider.columns)
print(cms_patient_provider.shape)

In [ ]:
cms_patient_provider["PotentialFraud"].value_counts()

### Merge with Provider data

#### Deriving claim duration

In [ ]:
## As patient can be admitted for only for 1 day,lets add 1 to the difference of Discharge Date and Admission Date 

cms_patient_provider['ClaimStartDt'] = pd.to_datetime(cms_patient_provider['ClaimStartDt'] , format = '%Y-%m-%d')
cms_patient_provider['ClaimEndDt'] = pd.to_datetime(cms_patient_provider['ClaimEndDt'],format = '%Y-%m-%d')
cms_patient_provider['ClaimDuration'] = ((cms_patient_provider['ClaimEndDt'] - cms_patient_provider['ClaimStartDt']).dt.days)+1

In [ ]:
cms_patient_provider.head(2)

In [ ]:

#cms_patient_provider.drop(["ClaimStartDt", "ClaimEndDt"], axis = 1, inplace=True)

In [ ]:
cms_patient_provider.head(2)


#### Let's check the missing columns

### No missing column as per the above code

In [ ]:
cms_patient_provider.columns

In [ ]:
### Count number of records
## From here we get the count of BeneID and ClaimId for each provider

## For Train 
cms_train_count = cms_patient_provider[['BeneID', 'ClaimID']].groupby(cms_patient_provider['Provider']).nunique().reset_index()
cms_train_count.rename(columns={'BeneID':'BeneID_count','ClaimID':'ClaimID_count'},inplace=True)

print(cms_train_count.head(5))


In [ ]:
#Gender will display Male count
cms_train_sum = cms_patient_provider.groupby(['Provider'], as_index = False)[['AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'IsInpatient',
        'MismatchedAdmitCode', 'Gender',
       'RenalDiseaseIndicator',
       'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke',
       'IsDead',
       'total_physicians', 'IsDiagnosisCode', 'IsDeductibleAmtPaid',
       'N_Procedure', 'N_UniqueDiagnosis_Claims','Age_Under_65', 'Age_65_upto_75', 
       'Age_75_upto_85', 'Age_85_or_Over','Race_1','Race_2','Race_3','Race_5']].sum()

print(cms_train_sum.head(5))



                                                  

In [ ]:
## Here we are calculating the mean of values for some variables for each unique provider.

cms_train_mean=round(cms_patient_provider.groupby(['Provider'], as_index = False)[['InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'AdmissionDuration', 
       'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
       'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 
       'NoOfMonths_PartACov', 'NoOfMonths_PartBCov']].mean())

print(cms_train_mean.head(3))

In [ ]:
#cms_patient_provider.drop(['State', 'County'], axis =1, inplace = True)

In [ ]:
## Merging of Train Datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
cms_train=pd.merge(cms_train_count,cms_train_sum,on='Provider',how='left').\
                merge(cms_train_mean,on='Provider',how='left').\
                merge(cms_provider, on = 'Provider', how='left')


In [ ]:

print(cms_train.head(5))

In [ ]:
### Check if categorical variables are present

In [ ]:

# store columns with specific data type
numeric_columns = cms_train.select_dtypes(include=['integer', 'float64', 'uint8']).columns
#float_columns = cms_train.select_dtypes(include=['float64']).columns
categorical_columns = cms_train.select_dtypes(include=['object']).columns

print("numeric_columns: ", numeric_columns)
print("categorical_columns :", categorical_columns)

### Target data analysis

In [ ]:
sns.countplot('PotentialFraud',data=cms_provider)

### Prepare Training & Test dataset 

In [ ]:
cms_test = cms_patient_provider[cms_patient_provider['PotentialFraud'] == "Test"]

In [ ]:
cms_test.shape

In [ ]:
cms_train = cms_train[(cms_train['PotentialFraud'] == "Yes")| (cms_train['PotentialFraud'] == "No") ]

In [2]:
cms_test.head(3)

NameError: name 'cms_test' is not defined

In [ ]:
cms_train = cms_train.replace({'PotentialFraud': 'Yes'}, 1)
cms_train = cms_train.replace({'PotentialFraud': 'No'}, 0)

In [ ]:
# change columns types - remove later
cms_train['PotentialFraud'] = cms_train['PotentialFraud'].apply(pd.to_numeric)
#churn_filtered[cat_cols] = churn_filtered[cat_cols].apply(lambda column: column.astype("category"), axis=0)

In [ ]:
cms_train.columns

In [ ]:
#provider_train
cms_train=cms_train.set_index('Provider')

In [ ]:
#Preparing training dataset as array - both X_train and y_train

X_prep = cms_train.drop(['PotentialFraud'],axis=1)
   

number_of_features = len(X_prep.columns)
X = X_prep.iloc[:,0:number_of_features].values

#Putting response variable to y
#y_prep = cms_train['PotentialFraud']


y = np.ravel(np.array(cms_train['PotentialFraud'].values))

In [ ]:
# Split into training and validation set
validation_size = 0.2
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=validation_size, random_state=seed)


In [ ]:
print(type(X_train))

In [ ]:
from sklearn.base import TransformerMixin
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
#from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [ ]:
scoring = 'accuracy'
# Model 3 - XGB Classifier
model_xgb = XGBClassifier()
model_xgb.fit(X_train, Y_train)
accuracy_score(Y_validation, model_xgb.predict(X_validation))

In [ ]:
# Model 1 - Logisitic Regression
model_logreg = LogisticRegression()
model_logreg.fit(X_train, Y_train)
print("LR:", accuracy_score(Y_validation, model_logreg.predict(X_validation)))

# Model 2 - RandomForest Classifier
model_rf = RandomForestClassifier()
model_rf.fit(X_train, Y_train)
print("RF : ", accuracy_score(Y_validation, model_rf.predict(X_validation)))

# Model 3 - XGB Classifier
model_xgb = XGBClassifier()
model_xgb.fit(X_train, Y_train)
print("XGB :", accuracy_score(Y_validation, model_xgb.predict(X_validation)))

#model_logreg = LogisticRegression()
#model_logreg.fit(X, y)

#model_rf = RandomForestClassifier()
#model_rf.fit(X, y)

#model_xgb = XGBClassifier()
#model_xgb.fit(X, y)

In [ ]:
!pip install lime

In [ ]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function

In [ ]:
predict_fn_logreg = lambda x: model_logreg.predict_proba(x).astype(float)
predict_fn_rf = lambda x: model_rf.predict_proba(x).astype(float)
predict_fn_xgb = lambda x: model_xgb.predict_proba(x).astype(float)



In [ ]:
# Line-up the feature names

feature_names = list(X_prep.columns)

In [ ]:

print(type(feature_names))
print(feature_names)

In [ ]:
cat_columns = X_prep.columns

In [ ]:
feature_names_cat = ['AttendingPhysician','OperatingPhysician', 'OtherPhysician', 'IsInpatient',
       'MismatchedAdmitCode', 'Gender', 'RenalDiseaseIndicator',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IsDead', 'IsDiagnosisCode',
       'IsDeductibleAmtPaid','Age_Under_65', 'Age_65_upto_75', 'Age_75_upto_85', 'Age_85_or_Over',
       'Race_1', 'Race_2', 'Race_3', 'Race_5', 'InscClaimAmtReimbursed']

In [ ]:
print(type(X_train))

In [ ]:
# Create the LIME Explainer
explainer = lime.lime_tabular.LimeTabularExplainer(X_train ,feature_names = feature_names,class_names=['0','1'],
                                                   categorical_features=cat_columns, 
                                                   categorical_names=feature_names_cat, kernel_width=3)

# Pick the observation in the validation set for which explanation is required
observation_1 = 2



In [ ]:
# Get the explanation for Logistic Regression
exp = explainer.explain_instance(X_validation[observation_1], predict_fn_logreg, num_features=6)
exp.show_in_notebook(show_all=False)

# Get the explanation for RandomForest
exp = explainer.explain_instance(X_validation[observation_1], predict_fn_rf, num_features=6)
exp.show_in_notebook(show_all=False)